이보시오, 그러면 힘들게 배운 머신러닝은 도대체 언제 써먹는단 말이오?

In [1]:
print('설무아쌤 웁니다')

설무아쌤 웁니다


![SortingHat](./image/SortingHat.jpg)

Magical... 아니고 MachineLearning Sorting Hat 은 어떨까?

학과 <- 학년, 반, 과목이수X5, 과목점수X5  
주어진 정보를 이용하여 신입생에게 학과를 배정해보자... 는 MSH project...  
1,1,2,1,2,2,2,2,5,3,5,3 이니까 그리핀도르... 아니고 전기과!

----------------
<span style="color:yellow">1. pandas 로 데이터 전처리 및 저장<span>

In [14]:
import pandas as pd
df19 = pd.read_csv('./data/1. 19년도-과정이수및평가점수.csv', encoding='euc-kr')

학생구분 은 학년 에 종속되어 결정되는 변수이므로 삭제하고 반 99 는 7로 변경한다.  
또한 비교적 분리가 쉬울 것으로 예상되는 전기과, 스포츠지도과, 아동문헌정보과 만 남긴 후 저장한다.

In [15]:
df19 = df19.replace({'반': {99: 7}}).drop('학생구분', axis=1)
df19 = df19[((df19['학과']=='전기과') | (df19['학과']=='스포츠지도과') | (df19['학과']=='아동문헌정보과'))]
df19.to_csv('./data/pdf19.csv', index=False, encoding='euc-kr')

In [17]:
pdf19 = pd.read_csv('./data/pdf19.csv', encoding='euc-kr')
pdf19

,학과,학년,반,A과목이수,B과목이수,C과목이수,D과목이수,E과목이수,A과목점수,B과목점수,C과목점수,D과목점수,E과목점수
0,스포츠지도과,1,1,2,2,2,2,2,4,6,5,5,4
1,스포츠지도과,1,1,2,2,2,2,2,5,5,0,3,3
2,스포츠지도과,1,1,2,2,2,2,2,2,4,3,3,2
3,스포츠지도과,1,1,2,2,2,2,2,2,5,3,4,4
4,스포츠지도과,1,1,2,2,2,2,2,3,6,4,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,전기과,3,2,2,2,1,1,2,4,5,5,4,5
325,전기과,3,2,2,2,1,2,2,2,7,6,7,5
326,전기과,3,2,2,1,1,2,2,2,7,6,7,5
327,전기과,3,2,2,2,2,1,1,1,3,3,7,4


In [30]:
print(pdf19[pdf19['학년']==1].shape)
print(pdf19[pdf19['학년']!=1].shape)

(177, 13)
(152, 13)


In [27]:
print(pdf19[pdf19['학과']=='전기과'].shape)
print(pdf19[(pdf19['학과']=='전기과') & (pdf19['학년']==1)].shape)
print(pdf19[(pdf19['학과']=='전기과') & (pdf19['학년']!=1)].shape)

(124, 13)
(76, 13)
(48, 13)


In [28]:
print(pdf19[pdf19['학과']=='스포츠지도과'].shape)
print(pdf19[(pdf19['학과']=='스포츠지도과') & (pdf19['학년']==1)].shape)
print(pdf19[(pdf19['학과']=='스포츠지도과') & (pdf19['학년']!=1)].shape)

(92, 13)
(39, 13)
(53, 13)


In [29]:
print(pdf19[pdf19['학과']=='아동문헌정보과'].shape)
print(pdf19[(pdf19['학과']=='아동문헌정보과') & (pdf19['학년']==1)].shape)
print(pdf19[(pdf19['학과']=='아동문헌정보과') & (pdf19['학년']!=1)].shape)

(113, 13)
(62, 13)
(51, 13)


전체 pdf19 데이터 중 20%를 테스트 세트로 만들고 나머지를 훈련 세트로 만든다.

In [38]:
pdf19_input = pdf19.drop('학과', axis=1).to_numpy()
pdf19_input

array([[1, 1, 2, ..., 5, 5, 4],
       [1, 1, 2, ..., 0, 3, 3],
       [1, 1, 2, ..., 3, 3, 2],
       ...,
       [3, 2, 2, ..., 6, 7, 5],
       [3, 2, 2, ..., 3, 7, 4],
       [3, 2, 1, ..., 6, 7, 5]], dtype=int64)

In [40]:
pdf19_target = pdf19['학과'].to_numpy()

In [42]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(pdf19_input, pdf19_target, test_size=0.2, random_state=33)

----------------
<span style="color:yellow">2. k-최근접 이웃 분류<span>

훈련 세트 중 다시 20%를 검증 세트로 만든다.

In [43]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=33)

In [45]:
print(sub_input.shape, val_input.shape)

(210, 12) (53, 12)


표준화 전처리

In [46]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(sub_input)
sub_scaled = ss.transform(sub_input)
val_scaled = ss.transform(val_input)

확률 예측

In [65]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=5)
kn.fit(sub_scaled, sub_target)
print(kn.score(sub_scaled, sub_target))
print(kn.score(val_scaled, val_target))

0.7857142857142857
0.6226415094339622


스코어는 걱정했던 것만큼 나쁘지는 않으나 과대적합된 상태이다.

In [66]:
kn.predict(val_scaled[:10])

array(['전기과', '아동문헌정보과', '스포츠지도과', '전기과', '아동문헌정보과', '전기과', '전기과', '전기과',
       '전기과', '전기과'], dtype=object)

In [67]:
val_target[:10]

array(['전기과', '아동문헌정보과', '스포츠지도과', '전기과', '아동문헌정보과', '스포츠지도과', '전기과',
       '전기과', '전기과', '아동문헌정보과'], dtype=object)

----------------
<span style="color:yellow">3. 로지스틱 회귀<span>

In [60]:
zip(kn.predict, val_target)

TypeError: 'method' object is not iterable